In [1]:
import numpy as np
import pandas as pd
import cx_Oracle
import os
import time
from datetime import datetime
from datetime import timedelta
from logging import *
from tkinter import messagebox

In [2]:
cwd = os.getcwd()
print(cwd)


C:\Users\Ashish.9650-IT-ASHISH


In [3]:
basicConfig(filename='error.log',filemode='a',format='%(linenos)s--%(asctime)s--%(message)s--%(porcess)s')

In [4]:
conn=cx_Oracle.connect('BSP_BF8/bspbf8@10.100.76.101:1521/BFL2')
print('connection done....')

connection done....


In [5]:
cursor8 = conn.cursor()

# Fetching interval and count values from another table (adjust the table and column names accordingly)
interval_query = f"SELECT ALARM_PERIOD FROM T_MESSAGE_DETAILS WHERE MSG_CODE = :msg_code"
description_query = f"SELECT MSG_DESCRIPTION FROM T_MESSAGE_DETAILS WHERE MSG_CODE = :msg_code"

# Set the MSG_CODE based on your requirement (1 for low, 2 for high, or any other value)
msg_code_low = 1
msg_code_high = 2

In [6]:
cursor8.execute(interval_query, {'msg_code': msg_code_low})
interval_value_low = cursor8.fetchone()[0]  # Assuming it returns a single value
print('interval_value_low:-', interval_value_low)

interval_value_low:- 60


In [7]:
# Fetching MSG_DESCRIPTION for low
cursor8.execute(description_query, {'msg_code': msg_code_low})
msg_description_low = cursor8.fetchone()[0]  # Assuming it returns a single value
print('msg_description_low:-', msg_description_low)

msg_description_low:- Lower than Low Limit


In [8]:
count_value_low = interval_value_low / 10
print('count_value_low:-', count_value_low)

count_value_low:- 6.0


In [9]:
# Fetching interval for high
cursor8.execute(interval_query, {'msg_code': msg_code_high})
interval_value_high = cursor8.fetchone()[0]  # Assuming it returns a single value
print('interval_value_high:-', interval_value_high)

interval_value_high:- 60


In [10]:
# Fetching MSG_DESCRIPTION for high
cursor8.execute(description_query, {'msg_code': msg_code_high})
msg_description_high = cursor8.fetchone()[0]  # Assuming it returns a single value
print('msg_description_high:-', msg_description_high)

msg_description_high:- Higher than High Limit


In [11]:
count_value_high = interval_value_high / 10
print('count_value_high:-', count_value_high)

count_value_high:- 6.0


In [12]:
# Forming the SQL query with the dynamic interval value for low
sql_low = f'''SELECT *
                        FROM T_ALARM_EVENT_MONITOR
                        WHERE MSG_CODE = {msg_code_low}
                        AND ALARM_TIME >= (SELECT MAX(ALARM_TIME) - INTERVAL '{interval_value_low}' MINUTE FROM T_ALARM_EVENT_MONITOR)
                        '''

In [13]:
# Forming the SQL query with the dynamic interval value for high
sql_high = f'''SELECT *
                        FROM T_ALARM_EVENT_MONITOR
                        WHERE MSG_CODE = {msg_code_high}
                        AND ALARM_TIME >= (SELECT MAX(ALARM_TIME) - INTERVAL '{interval_value_high}' MINUTE FROM T_ALARM_EVENT_MONITOR)
                        '''

In [14]:
# Fetching data for low
cursor8.execute(sql_low)
fetched_data_low = cursor8.fetchall()
fetched_column_low = [col[0] for col in cursor8.description]
df_alarm_low = pd.DataFrame(fetched_data_low, columns=fetched_column_low)

In [15]:
print('df_alarm_low')
print(df_alarm_low.head(20))
print('df_alarm_low:-',df_alarm_low.shape)

df_alarm_low
   S_NO          ALARM_TIME  TAG_ID                  TAG_NAME  TAG_VALUE  \
0   338 2023-06-13 18:10:00   92012  L2_AVG_HEARTH_DEL_TEMP_D      7.370   
1   378 2023-06-13 18:30:00   92012  L2_AVG_HEARTH_DEL_TEMP_D      7.388   
2   398 2023-06-13 18:40:00   92012  L2_AVG_HEARTH_DEL_TEMP_D      7.389   
3   358 2023-06-13 18:20:00   92012  L2_AVG_HEARTH_DEL_TEMP_D      7.384   
4   418 2023-06-13 18:50:00   92012  L2_AVG_HEARTH_DEL_TEMP_D      7.395   

   MSG_CODE  ALARM_STATUS  
0         1             1  
1         1             1  
2         1             1  
3         1             1  
4         1             1  
df_alarm_low:- (5, 7)


In [16]:
# Fetching data for high
cursor8.execute(sql_high)
fetched_data_high = cursor8.fetchall()
fetched_column_high = [col[0] for col in cursor8.description]
df_alarm_high = pd.DataFrame(fetched_data_high, columns=fetched_column_high)
print('df_alarm_high')
print(df_alarm_high.head(20))
print('df_alarm_high:-',df_alarm_high.shape)

df_alarm_high
    S_NO          ALARM_TIME  TAG_ID                TAG_NAME  TAG_VALUE  \
0    319 2023-06-13 18:10:00    8134  HEARTH_TEMP_K_VL03_TC1     47.915   
1    320 2023-06-13 18:10:00    8160  HEARTH_TEMP_P_VL01_TC1    104.935   
2    321 2023-06-13 18:10:00    8161  HEARTH_TEMP_P_VL02_TC1    109.751   
3    322 2023-06-13 18:10:00    8162  HEARTH_TEMP_P_VL03_TC1    106.850   
4    323 2023-06-13 18:10:00    8163  HEARTH_TEMP_P_VL04_TC1    110.584   
5    324 2023-06-13 18:10:00    8164  HEARTH_TEMP_P_VL05_TC1    109.132   
6    325 2023-06-13 18:10:00    8165  HEARTH_TEMP_P_VL06_TC1    103.096   
7    326 2023-06-13 18:10:00    8184  HEARTH_TEMP_P_VL25_TC1    110.343   
8    327 2023-06-13 18:10:00    8185  HEARTH_TEMP_P_VL26_TC1    106.052   
9    328 2023-06-13 18:10:00    8186  HEARTH_TEMP_P_VL27_TC1    105.657   
10   329 2023-06-13 18:10:00    8187  HEARTH_TEMP_P_VL28_TC1    103.944   
11   330 2023-06-13 18:10:00    8188  HEARTH_TEMP_P_VL29_TC1     98.258   
12   331 20

In [17]:
# Adding MSG_DESCRIPTION column to low DataFrame
df_alarm_low['MSG_DESCRIPTION'] = msg_description_low

# Adding MSG_DESCRIPTION column to high DataFrame
df_alarm_high['MSG_DESCRIPTION'] = msg_description_high

In [18]:
# Filtered DataFrame for low
filtered_df_alarm_low = df_alarm_low[df_alarm_low['MSG_CODE'] == msg_code_low]
print('filtered_df_alarm_low')
print(filtered_df_alarm_low.head(6))
print('filtered_df_alarm_low:-' ,filtered_df_alarm_low.shape)

filtered_df_alarm_low
   S_NO          ALARM_TIME  TAG_ID                  TAG_NAME  TAG_VALUE  \
0   338 2023-06-13 18:10:00   92012  L2_AVG_HEARTH_DEL_TEMP_D      7.370   
1   378 2023-06-13 18:30:00   92012  L2_AVG_HEARTH_DEL_TEMP_D      7.388   
2   398 2023-06-13 18:40:00   92012  L2_AVG_HEARTH_DEL_TEMP_D      7.389   
3   358 2023-06-13 18:20:00   92012  L2_AVG_HEARTH_DEL_TEMP_D      7.384   
4   418 2023-06-13 18:50:00   92012  L2_AVG_HEARTH_DEL_TEMP_D      7.395   

   MSG_CODE  ALARM_STATUS       MSG_DESCRIPTION  
0         1             1  Lower than Low Limit  
1         1             1  Lower than Low Limit  
2         1             1  Lower than Low Limit  
3         1             1  Lower than Low Limit  
4         1             1  Lower than Low Limit  
filtered_df_alarm_low:- (5, 8)


In [19]:
# Filtered DataFrame for high
filtered_df_alarm_high = df_alarm_high[df_alarm_high['MSG_CODE'] == msg_code_high]
print('filtered_df_alarm_high')
print(filtered_df_alarm_high.head(6))
print('filtered_df_alarm_high:-' ,filtered_df_alarm_high.shape)

filtered_df_alarm_high
   S_NO          ALARM_TIME  TAG_ID                TAG_NAME  TAG_VALUE  \
0   319 2023-06-13 18:10:00    8134  HEARTH_TEMP_K_VL03_TC1     47.915   
1   320 2023-06-13 18:10:00    8160  HEARTH_TEMP_P_VL01_TC1    104.935   
2   321 2023-06-13 18:10:00    8161  HEARTH_TEMP_P_VL02_TC1    109.751   
3   322 2023-06-13 18:10:00    8162  HEARTH_TEMP_P_VL03_TC1    106.850   
4   323 2023-06-13 18:10:00    8163  HEARTH_TEMP_P_VL04_TC1    110.584   
5   324 2023-06-13 18:10:00    8164  HEARTH_TEMP_P_VL05_TC1    109.132   

   MSG_CODE  ALARM_STATUS         MSG_DESCRIPTION  
0         2             1  Higher than High Limit  
1         2             1  Higher than High Limit  
2         2             1  Higher than High Limit  
3         2             1  Higher than High Limit  
4         2             1  Higher than High Limit  
5         2             1  Higher than High Limit  
filtered_df_alarm_high:- (133, 8)


In [20]:
# Grouped DataFrame for low
grouped_df_alarm_low = filtered_df_alarm_low.groupby('TAG_ID').count()
print('grouped_df_alarm_low:-', grouped_df_alarm_low.head())
print('grouped_df_alarm_low:-',grouped_df_alarm_low.shape)

grouped_df_alarm_low:-         S_NO  ALARM_TIME  TAG_NAME  TAG_VALUE  MSG_CODE  ALARM_STATUS  \
TAG_ID                                                                  
92012      5           5         5          5         5             5   

        MSG_DESCRIPTION  
TAG_ID                   
92012                 5  
grouped_df_alarm_low:- (1, 7)


In [21]:
# Grouped DataFrame for high
grouped_df_alarm_high = filtered_df_alarm_high.groupby('TAG_ID').count()
print('grouped_df_alarm_high:-', grouped_df_alarm_high.head())
print('grouped_df_alarm_high:-',grouped_df_alarm_high.shape)

grouped_df_alarm_high:-         S_NO  ALARM_TIME  TAG_NAME  TAG_VALUE  MSG_CODE  ALARM_STATUS  \
TAG_ID                                                                  
8134       7           7         7          7         7             7   
8160       7           7         7          7         7             7   
8161       7           7         7          7         7             7   
8162       7           7         7          7         7             7   
8163       7           7         7          7         7             7   

        MSG_DESCRIPTION  
TAG_ID                   
8134                  7  
8160                  7  
8161                  7  
8162                  7  
8163                  7  
grouped_df_alarm_high:- (19, 7)


In [22]:
# Selected tags for low
selected_tags_low = grouped_df_alarm_low[grouped_df_alarm_low['S_NO'] >= count_value_low].index
# Selected tags for high
selected_tags_high = grouped_df_alarm_high[grouped_df_alarm_high['S_NO'] >= count_value_high].index

In [23]:
# Result DataFrame for low
result_df_alarm_low = df_alarm_low[df_alarm_low['TAG_ID'].isin(selected_tags_low)].groupby('TAG_ID').last()
print('result_df_alarm_low')
print(result_df_alarm_low.head(20))
print('result_df_alarm_low:-',result_df_alarm_low.shape)

result_df_alarm_low
Empty DataFrame
Columns: [S_NO, ALARM_TIME, TAG_NAME, TAG_VALUE, MSG_CODE, ALARM_STATUS, MSG_DESCRIPTION]
Index: []
result_df_alarm_low:- (0, 7)


In [24]:
# Result DataFrame for high
result_df_alarm_high = df_alarm_high[df_alarm_high['TAG_ID'].isin(selected_tags_high)].groupby('TAG_ID').last()
print('result_df_alarm_high')
print(result_df_alarm_high.head(20))
print('result_df_alarm_high:-',result_df_alarm_high.shape)

result_df_alarm_high
        S_NO          ALARM_TIME                TAG_NAME  TAG_VALUE  MSG_CODE  \
TAG_ID                                                                          
8134     399 2023-06-13 18:50:00  HEARTH_TEMP_K_VL03_TC1     48.086         2   
8160     400 2023-06-13 18:50:00  HEARTH_TEMP_P_VL01_TC1    102.633         2   
8161     401 2023-06-13 18:50:00  HEARTH_TEMP_P_VL02_TC1    107.008         2   
8162     402 2023-06-13 18:50:00  HEARTH_TEMP_P_VL03_TC1    103.033         2   
8163     403 2023-06-13 18:50:00  HEARTH_TEMP_P_VL04_TC1    107.636         2   
8164     404 2023-06-13 18:50:00  HEARTH_TEMP_P_VL05_TC1    106.409         2   
8165     405 2023-06-13 18:50:00  HEARTH_TEMP_P_VL06_TC1    100.775         2   
8184     406 2023-06-13 18:50:00  HEARTH_TEMP_P_VL25_TC1    109.715         2   
8185     407 2023-06-13 18:50:00  HEARTH_TEMP_P_VL26_TC1    104.890         2   
8186     408 2023-06-13 18:50:00  HEARTH_TEMP_P_VL27_TC1    103.210         2   
8187   

In [25]:
# Concatenate the result DataFrames
result_df_alarm = pd.concat([result_df_alarm_low, result_df_alarm_high])
result_df_alarm.reset_index(inplace=True,drop=False)

In [26]:
print('result_df_alarm:-', result_df_alarm.head())
print(result_df_alarm.tail())
print(result_df_alarm.shape)

result_df_alarm:-    TAG_ID  S_NO          ALARM_TIME                TAG_NAME  TAG_VALUE  \
0    8134   399 2023-06-13 18:50:00  HEARTH_TEMP_K_VL03_TC1     48.086   
1    8160   400 2023-06-13 18:50:00  HEARTH_TEMP_P_VL01_TC1    102.633   
2    8161   401 2023-06-13 18:50:00  HEARTH_TEMP_P_VL02_TC1    107.008   
3    8162   402 2023-06-13 18:50:00  HEARTH_TEMP_P_VL03_TC1    103.033   
4    8163   403 2023-06-13 18:50:00  HEARTH_TEMP_P_VL04_TC1    107.636   

   MSG_CODE  ALARM_STATUS         MSG_DESCRIPTION  
0         2             1  Higher than High Limit  
1         2             1  Higher than High Limit  
2         2             1  Higher than High Limit  
3         2             1  Higher than High Limit  
4         2             1  Higher than High Limit  
    TAG_ID  S_NO          ALARM_TIME                TAG_NAME  TAG_VALUE  \
14    8191   413 2023-06-13 18:50:00  HEARTH_TEMP_P_VL32_TC1    106.393   
15    8192   414 2023-06-13 18:50:00  HEARTH_TEMP_P_VL33_TC1     83.646   


In [27]:
cursor9 = conn.cursor()
S_NO_sql = f'SELECT MAX(S_NO) FROM T_ALARM_HMI_MONITOR'
                
cursor9.execute(S_NO_sql)
result = cursor9.fetchone()

# Check if the result is not None
if result is not None and result[0] is not None:
    max_s_no = result[0]
else:
    max_s_no = 0  # Set a default value if no results are returned
# Start the serial numbers from the next number after the maximum serial number of inserted data in
result_df_alarm['SL_NO'] = range(max_s_no + 1, (max_s_no + 1) + len(result_df_alarm))

In [28]:
for index, row in result_df_alarm.iterrows():
    max_alarm_time = np.max(result_df_alarm['ALARM_TIME']).to_pydatetime()

In [29]:
#Insert data into the T_ALARM_EVENT_MONITOR
cursor10 = conn.cursor()
cursor10.execute(f"""INSERT INTO T_ALARM_HMI_MONITOR
                        (S_NO,TAG_NAME, ALARM_DT, TAG_ID, TAG_VALUE, MSG_CODE,MSG_DESCRIPTION,ALARM_STATUS)
                        VALUES (:1, :2, :3, :4, :5,:6,:7,:8)
                    """, (row['SL_NO'], row['TAG_NAME'],max_alarm_time, row['TAG_ID'], row['TAG_VALUE'], row['MSG_CODE'],row['MSG_DESCRIPTION'],row['ALARM_STATUS']))
conn.commit()
cursor10.close()
print('''Log Data inserted ......''')

Log Data inserted ......
